In [36]:
# JUPYTER NOTEBOOK: MCQ decoding + parsing sanity test for vLLM + Jamba/Mamba/etc.

import re
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"
os.environ["VLLM_ALLOW_LONG_MAX_MODEL_LEN"] = "1"

from typing import Optional, List, Dict, Any
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

# -----------------------
# Config (edit these)
# -----------------------
MODEL = "ibm-ai-platform/Bamba-9B-v1"   # or "ai21labs/Jamba-7B", "mistralai/Mamba-Codestral-7B-v0.1", etc.
CTX_LEN = 16384
DTYPE = "float16"
TP = 1
GPU_MEM_UTIL = 0.90

USE_CHAT = True   # for Jamba; set False for some completion-only models
MAX_NEW_MAIN = 4  # MCQ: 2-4 is plenty
MAX_NEW_REP = 1

# -----------------------
# Prompt builder
# -----------------------
DEFAULT_0SHOT = """{doc}

Question: {q}

A. {a}
B. {b}
C. {c}
D. {d}

Answer:"""

def build_prompt(doc: str, q: str, A: str, B: str, C: str, D: str) -> str:
    p = DEFAULT_0SHOT.format(doc=doc.strip(), q=q.strip(), a=A.strip(), b=B.strip(), c=C.strip(), d=D.strip())
    p += "\n\nReturn exactly one letter: A, B, C, or D."
    return p

def should_use_chat(name: str) -> bool:
    s = (name or "").lower()
    return any(k in s for k in ["instruct", "chat", "assistant", "jamba"])

def maybe_apply_chat_template(tok, prompt: str, use_chat: bool) -> str:
    if not use_chat:
        return prompt
    if hasattr(tok, "apply_chat_template"):
        return tok.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True,
        )
    return prompt

# -----------------------
# Answer extractor (fixed)
# -----------------------
ANSWER_PATTERNS = [
    re.compile(r"(?:final\s*answer|answer)\s*[:\-]\s*\(?([A-D])\)?", re.IGNORECASE),
    re.compile(r"the\s+correct\s+answer\s+is\s*\(?([A-D])\)?", re.IGNORECASE),
]

def extract_answer(response: str) -> Optional[str]:
    if not response:
        return None
    text = response.replace("*", "").strip()
    tail = text[-512:]
    for pat in ANSWER_PATTERNS:
        m = pat.search(tail)
        if m:
            return m.group(1).upper()
    m = re.search(r"\b([A-D])\b", tail)
    return m.group(1).upper() if m else None

# -----------------------
# Load tokenizer + vLLM
# -----------------------

llm = LLM(
    model=MODEL,
    trust_remote_code=True,
    dtype=DTYPE,
    tensor_parallel_size=TP,
    max_model_len=CTX_LEN,
    gpu_memory_utilization=GPU_MEM_UTIL,
)




tests: List[Dict[str, Any]] = [
    {
        "id": "toy_1",
        "doc": "Video PreTraining (VPT) learns to act by watching unlabeled videos and uses a small labeled set to train an inverse dynamics model to label actions.",
        "q": "Which of the following statements is correct?",
        "A": "Both contractor data and data crawled from the Internet are used to train VPT agents to model state-action pairs.",
        "B": "All machine learning methods involved in the two articles are related to neural network deep learning.",
        "C": "Both voyager and VPT control Minecraft agents by predicting the actions of simulated mouse and keyboard operations in each given state.",
        "D": "VPT's modeling of action space is approximate rather than precise.",
        "gold": "D",
    },
]


INFO 12-17 07:27:23 [utils.py:233] non-default args: {'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 16384, 'disable_log_stats': True, 'model': 'ibm-ai-platform/Bamba-9B-v1'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-17 07:27:24 [model.py:547] Resolved architecture: BambaForCausalLM


Parse safetensors files: 100%|████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 16.06it/s]

WARNING 12-17 07:27:24 [model.py:1907] User-specified max_model_len (16384) is greater than the derived max_model_len (max_position_embeddings=4096 or model_max_length=None in model's config.json). VLLM_ALLOW_LONG_MAX_MODEL_LEN must be used with extreme caution. If the model uses relative position encoding (RoPE), positions exceeding derived_max_model_len lead to nan. If the model uses absolute position encoding, positions exceeding derived_max_model_len will cause a CUDA array out-of-bounds error.
INFO 12-17 07:27:24 [model.py:1510] Using max model len 16384
INFO 12-17 07:27:24 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 12-17 07:27:24 [config.py:297] Hybrid or mamba-based model detected: disabling prefix caching since it is not yet supported.
INFO 12-17 07:27:24 [config.py:308] Hybrid or mamba-based model detected: setting cudagraph mode to FULL_AND_PIECEWISE in order to optimize performance.
INFO 12-17 07:27:24 [config.py:376] Setting attenti

...

(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=32, uniform_decode=True))
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=24, uniform_decode=True))
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=16, uniform_decode=True))
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=8, uniform_decode=True))


Capturing CUDA graphs (decode, FULL):  97%|█████████▋| 34/35 [00:03<00:00, 15.35it/s]

(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=4, uniform_decode=True))
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=2, uniform_decode=True))
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:35 [compilation/cuda_graph.py:136] Capturing a cudagraph on (FULL,BatchDescriptor(num_tokens=1, uniform_decode=True))


Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:04<00:00,  7.95it/s]


(EngineCore_DP0 pid=2290876) INFO 12-17 07:28:36 [v1/worker/gpu_model_runner.py:3480] Graph capturing finished in 12 secs, took 0.68 GiB
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:36 [v1/worker/gpu_worker.py:393] Free memory on device (47.1/47.4 GiB) on startup. Desired GPU memory utilization is (0.9, 42.66 GiB). Actual usage is 18.25 GiB for weight, 1.08 GiB for peak activation, 0.04 GiB for non-torch memory, and 0.68 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=24133454950` (22.48 GiB) to fit into requested memory, or `--kv-cache-memory=28894258688` (26.91 GiB) to fully utilize gpu memory. Current kv cache memory in use is 23.3 GiB.
(EngineCore_DP0 pid=2290876) INFO 12-17 07:28:36 [v1/engine/core.py:210] init engine (profile, create kv cache, warmup model) took 30.15 seconds
(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:28:37 [v1/engine/core.py:737] EngineCore waiting for work.
INFO 12-17 07:28:37 [llm.py:306] Supported_tasks: ['generate']

In [40]:
import re
from typing import Optional, List, Dict, Any
from transformers import AutoTokenizer
from vllm import SamplingParams

# -------------------------
# Config for BAMBA
# -------------------------
MODEL = "ibm-ai-platform/Bamba-9B-v1"  # adjust if your hf_id differs
USE_CHAT = False  # IMPORTANT: Bamba is completion-style
MAX_NEW_MAIN = 8
MAX_NEW_REP = 16

# -------------------------
# Answer patterns + extractor
# -------------------------
ANSWER_PATTERNS = [
    re.compile(r"(?:final\s*answer|answer)\s*[:\-]\s*\(?\s*([A-D])\s*\)?", re.IGNORECASE),
    re.compile(r"the\s+correct\s+answer\s+is\s*[:\-]?\s*\(?\s*([A-D])\s*\)?", re.IGNORECASE),
]

def extract_answer(response: str) -> Optional[str]:
    if not response:
        return None
    text = response.replace("*", "").strip()
    tail = text[-512:]

    m = re.search(r"\\boxed\s*\{\s*([A-D])\s*\}", tail, re.IGNORECASE)
    if m:
        return m.group(1).upper()

    for pat in ANSWER_PATTERNS:
        m = pat.search(tail)
        if m:
            return m.group(1).upper()

    m = re.search(r"\b([A-D])\b", tail)
    return m.group(1).upper() if m else None

# -------------------------
# Prompt builders
# -------------------------
DEFAULT_0SHOT = """{doc}

Question: {q}

A. {a}
B. {b}
C. {c}
D. {d}

Final answer:"""

def add_filler_tokens(prompt: str, tok, n_tokens: int = 5000) -> str:
    """
    Appends ~n_tokens of harmless filler text, measured in tokenizer tokens.
    """
    filler_unit = " lorem"  # stable, non-special
    unit_ids = tok.encode(filler_unit, add_special_tokens=False)
    if not unit_ids:
        raise RuntimeError("Tokenizer produced no tokens for filler_unit")

    reps = (n_tokens // len(unit_ids)) + 1
    filler_text = filler_unit * reps

    # Trim to exactly n_tokens
    ids = tok.encode(filler_text, add_special_tokens=False)[:n_tokens]
    filler_text = tok.decode(ids, skip_special_tokens=True)

    return prompt + "\n\n[FILLER]\n" + filler_text

def build_prompt(doc: str, q: str, A: str, B: str, C: str, D: str) -> str:
    return DEFAULT_0SHOT.format(
        doc=doc.strip(), q=q.strip(), a=A.strip(), b=B.strip(), c=C.strip(), d=D.strip()
    )

def build_reprompt(q: str, A: str, B: str, C: str, D: str, prev: str) -> str:
    # Keep it short; include choices so A/B/C/D is grounded.
    return (
        "Return exactly one letter: A, B, C, or D.\n\n"
        f"Question: {q.strip()}\n"
        f"Prev: {prev.strip()[:300]}\n"
        f"A. {A.strip()}\n"
        f"B. {B.strip()}\n"
        f"C. {C.strip()}\n"
        f"D. {D.strip()}\n\n"
        "Final answer:"
    )

# -------------------------
# Tokenizer + sampling params
# -------------------------
tok = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

sp_main = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=MAX_NEW_MAIN, seed=0)
sp_rep  = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=MAX_NEW_REP, seed=42)

# -------------------------
# Run tests (expects `llm` and `tests` already defined)
# tests items must have: id, doc, q, A, B, C, D, (optional) gold
# -------------------------
for t in tests:
    raw = build_prompt(t["doc"], t["q"], t["A"], t["B"], t["C"], t["D"])

    # IMPORTANT: vLLM expects a list of prompts
    raw = add_filler_tokens(raw, tok, n_tokens=5000)

    out = llm.generate([raw], sp_main)[0]
    text = out.outputs[0].text if out.outputs else ""
    pred = extract_answer(text)

    rep_text = ""
    if pred is None:
        rep_raw = build_reprompt(t["q"], t["A"], t["B"], t["C"], t["D"], text)
        print(rep_raw)
        rep_out = llm.generate([rep_raw], sp_rep)[0]
        rep_text = rep_out.outputs[0].text if rep_out.outputs else ""
        pred = extract_answer(rep_text)

    print("=" * 80)
    print("id:", t["id"])
    print("PROMPT (last 400 chars):\n", raw[-400:])
    print("\nMAIN OUTPUT:\n", repr(text))
    print("MAIN pred:", pred)
    if rep_text:
        print("\nREPROMPT OUTPUT:\n", repr(rep_text))
        print("REP pred:", pred)
    if "gold" in t:
        print("gold:", t["gold"], "correct:", (pred == t["gold"]))



Adding requests: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 28.23it/s]


(EngineCore_DP0 pid=2290876) DEBUG 12-17 07:34:08 [v1/engine/core.py:743] EngineCore loop active.


Processed prompts:   0%|                  | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

(EngineCore_DP0 pid=2290876) ERROR 12-17 07:34:09 [logging_utils/dump_input.py:69] Dumping input data for V1 LLM engine (v0.11.0) with config: model='ibm-ai-platform/Bamba-9B-v1', speculative_config=None, tokenizer='ibm-ai-platform/Bamba-9B-v1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_model_name=ibm-ai-platform/Bamba-9B-v1, enable_p

/insomnia001/home/dwz2107/.cache/vllm/torch_compile_cache/66dcbb55ec/rank_0_0/inductor_cache/ii/ciil7rhth7sqj5ecr2awv7wlwey2bil2uiejvrqlufo3xiowouz7.py:37: unknown: block: [8338,0,0], thread: [64,0,0] Assertion `index out of bounds: 0 <= tl.broadcast_to(tmp10, [XBLOCK]) < 4096` failed.
/insomnia001/home/dwz2107/.cache/vllm/torch_compile_cache/66dcbb55ec/rank_0_0/inductor_cache/ii/ciil7rhth7sqj5ecr2awv7wlwey2bil2uiejvrqlufo3xiowouz7.py:37: unknown: block: [8338,0,0], thread: [65,0,0] Assertion `index out of bounds: 0 <= tl.broadcast_to(tmp10, [XBLOCK]) < 4096` failed.
/insomnia001/home/dwz2107/.cache/vllm/torch_compile_cache/66dcbb55ec/rank_0_0/inductor_cache/ii/ciil7rhth7sqj5ecr2awv7wlwey2bil2uiejvrqlufo3xiowouz7.py:37: unknown: block: [8338,0,0], thread: [66,0,0] Assertion `index out of bounds: 0 <= tl.broadcast_to(tmp10, [XBLOCK]) < 4096` failed.
/insomnia001/home/dwz2107/.cache/vllm/torch_compile_cache/66dcbb55ec/rank_0_0/inductor_cache/ii/ciil7rhth7sqj5ecr2awv7wlwey2bil2uiejvrqlufo

EngineDeadError: EngineCore encountered an issue. See stack trace (above) for the root cause.

[rank0]:[W1217 07:34:09.862728493 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())
